# Introduction

- Provide a project overview and a clear problem description, including a history of how the problem came about and why the problem is important/interesting.
- Describe how you acquired the data and provide a concise summary of any cleaning steps taken (i.e., summarize how you cleaned the data in the  pre-processing notebook).
- Give an outline of the rest of the report and list any key assumptions made to simplify problems.
- Briefly summarize your overall approach.

# Descriptive Analysis

- Load the (cleaned) dataset into a Pandas DataFrame. 
- Use at least one method (e.g., Pivot Tables, PCA, Clustering) to provide a descriptive analysis of the data. 
- Include at least one chart to enhance your analysis. 
- Describe any insight you gained from your analysis. Are there any surprises?
- Use Markdown cells to provide written analysis where appropriate. 

# Predictive Analysis

- Use a markdown cell to identify at least one question that can be addressed via predictive analysis. (E.g., “Can house prices be predicted based on the year they were built and the size of its lot?”)
- Implement an appropriate supervised learning algorithm to address the question(s) identified above, explaining your choice of algorithm over other methods and specifying the features and labels of your model(s).
- Include a ML pipeline with appropriate training/testing splits and, when applicable, cross-validation and hyperparameter tuning.
- Report your results using appropriate metrics and visualizations.
- Use Markdown cells to provide a written analysis.

The Question we aim to address through our predictive analysis is whether we can

# Ethical Considerations

**Are there any ethical considerations regarding the dataset? (e.g., is it possible the data is biased?)**\
The Team Innovation Lab isn’t open to students on the weekends so the only prints on the weekend are staff. So there is some bias towards staff as they can print throughout the whole week versus students who can only print five days a week.

**Reflect on potential ethical implications of your analysis, such as privacy concerns, fairness, and the societal impact of the model if misused.**\
The Team Innovation Lab isn’t open to students on the weekends so the only prints on the weekend are staff. So there is some bias towards staff as they can print throughout the whole week versus students who can only print five days a week.

**Address ways to mitigate negative impacts of the analysis.**\
We have chosen to not use names after cleaning to avoid privacy issues that could be associated with them. We are just showing our findings in class and to the Team Innovations Lab instead of to the public to mitigate potential copying of ideas.

# Conclusion

- Use a Markdown cell to write a concise conclusion to your analysis. Did your analysis provide insight to this dataset? Were the results expected? If not, why?
- Highlight any limitations encountered.
- Describe at least one possible future direction based on your analysis and dataset. What are the challenges of pursuing this direction? If additional resources/data is needed, please specify.